In [1]:
# Install dependencies (uncomment if needed)
# !pip install yfinance pandas numpy statsmodels matplotlib seaborn PyPortfolioOpt

# Imports
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting

# Define assets
asset_tickers = [
    "AIA.AX", "ALL.AX", "AMC.AX", "ANZ.AX", "APA.AX", "ASX.AX",
    "BHP.AX", "BSL.AX", "BXB.AX", "CBA.AX", "COH.AX", "COL.AX", "CSL.AX",
    "DXS.AX", "FMG.AX", "FPH.AX", "GMG.AX", "IAG.AX", "JHX.AX", "MGOC.AX",
    "MGR.AX", "MQG.AX", "NAB.AX", "NST.AX", "QBE.AX", "REA.AX",
    "REH.AX", "RHC.AX", "RIO.AX", "RMD.AX", "S32.AX", "SCG.AX", "SEK.AX",
    "SGP.AX", "SHL.AX", "STO.AX", "SUN.AX", "TAH.AX", "TCL.AX",
    "TLS.AX", "TPG.AX", "WBC.AX", "WES.AX", "WOW.AX", "WTC.AX", "XRO.AX"
]
market_ticker = "^AXJO"

# Download market index
market_data = yf.download(market_ticker, start="2018-01-01", end="2023-12-31", auto_adjust=True)["Adj Close"]
market_returns = market_data.pct_change(fill_method=None).dropna()

# Download asset prices
asset_data = yf.download(asset_tickers, start="2018-01-01", end="2023-12-31", auto_adjust=True)["Adj Close"]
asset_returns = asset_data.pct_change(fill_method=None).dropna()

# align dates
common_dates = asset_returns.index.intersection(market_returns.index)
asset_returns = asset_returns.loc[common_dates]
market_returns = market_returns.loc[common_dates]

# check missing columns
missing = set(asset_tickers) - set(asset_returns.columns)
if missing:
    print(f"⚠️ Missing tickers with no data: {missing}")

# CAPM analysis
results = []

risk_free_rate = 0.03
expected_market_return = 0.08

for ticker in asset_returns.columns:
    y = asset_returns[ticker]
    X = sm.add_constant(market_returns)
    model = sm.OLS(y, X).fit()
    alpha = model.params[0]
    beta = model.params[1]
    expected_return = risk_free_rate + beta * (expected_market_return - risk_free_rate)

    results.append({
        "Ticker": ticker,
        "Alpha": alpha,
        "Beta": beta,
        "Expected Return (CAPM)": expected_return
    })

capm_summary = pd.DataFrame(results)
capm_summary

# Plot Security Market Line
betas = np.linspace(0, 2, 20)
sml_returns = risk_free_rate + betas * (expected_market_return - risk_free_rate)

plt.figure(figsize=(10, 6))
plt.plot(betas, sml_returns, label="Security Market Line")

for _, row in capm_summary.iterrows():
    plt.scatter(row['Beta'], row['Expected Return (CAPM)'], label=row['Ticker'])

plt.xlabel("Beta")
plt.ylabel("Expected Return")
plt.title("Security Market Line with Assets")
plt.legend(fontsize="small", ncol=2)
plt.grid(True)
plt.show()

# Portfolio optimizer
mu = expected_returns.mean_historical_return(asset_data)
S = risk_models.sample_cov(asset_data)

ef = EfficientFrontier(mu, S)
ef.add_constraint(lambda w: w >= 0)
ef.add_constraint(lambda w: w[np.where(w > 0)[0]].shape[0] <= 5)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("✅ Optimal Portfolio Weights (max 5 assets):")
print(cleaned_weights)

performance = ef.portfolio_performance(verbose=True)

# Efficient Frontier
ef_plot = EfficientFrontier(mu, S)
plotting.plot_efficient_frontier(ef_plot, show_assets=True, show_fig=True)

# Done!

[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj Close'

In [ ]:

# Define assets
asset_tickers = [
    "AIA.AX", "ALL.AX", "AMC.AX", "ANZ.AX", "APA.AX", "ASX.AX",
    "BHP.AX", "BSL.AX", "BXB.AX", "CBA.AX", "COH.AX", "COL.AX", "CSL.AX",
    "DXS.AX", "FMG.AX", "FPH.AX", "GMG.AX", "IAG.AX", "JHX.AX", "MGOC.AX",
    "MGR.AX", "MQG.AX", "NAB.AX", "NST.AX", "QBE.AX", "REA.AX",
    "REH.AX", "RHC.AX", "RIO.AX", "RMD.AX", "S32.AX", "SCG.AX", "SEK.AX",
    "SGP.AX", "SHL.AX", "STO.AX", "SUN.AX", "TAH.AX", "TCL.AX",
    "TLS.AX", "TPG.AX", "WBC.AX", "WES.AX", "WOW.AX", "WTC.AX",
    "XRO.AX"
]
market_ticker = "^AXJO"

# 1. download market index separately
market_data = yf.download(market_ticker,
                          start="2018-01-01",
                          end="2023-12-31",
                          auto_adjust=False)["Adj Close"]

market_returns = market_data.pct_change(fill_method=None).dropna()

# 2. download asset prices
asset_data = yf.download(asset_tickers,
                         start="2018-01-01",
                         end="2023-12-31",
                         auto_adjust=False)["Adj Close"]

# 3. calculate returns
asset_returns = asset_data.pct_change(fill_method=None).dropna()

# 4. align with market index
common_dates = asset_returns.index.intersection(market_returns.index)
asset_returns = asset_returns.loc[common_dates]
market_returns = market_returns.loc[common_dates]

# 5. check for missing columns
missing_cols = set(asset_tickers) - set(asset_returns.columns)
if missing_cols:
    print(f" Warning: These tickers had missing data and were dropped: {missing_cols}")

# 6. preview
print("Loaded assets:", asset_returns.columns.tolist())
print("Market index loaded:", market_ticker)

# Loop through to calc CAPM stats

In [ ]:
# Calculate CAPM stats with correct returns
results = []

# reuse market_returns from your cleaned loader
# reuse asset_returns from your cleaned loader

risk_free_rate = 0.03
expected_market_return = 0.08

for ticker in asset_returns.columns:
    y = asset_returns[ticker]
    X = sm.add_constant(market_returns)
    model = sm.OLS(y, X).fit()
    alpha = model.params[0]
    beta = model.params[1]
    expected_return = risk_free_rate + beta * (expected_market_return - risk_free_rate)

    results.append({
        "Ticker": ticker,
        "Alpha": alpha,
        "Beta": beta,
        "Expected Return (CAPM)": expected_return
    })

capm_summary = pd.DataFrame(results)
capm_summary

In [ ]:
betas = np.linspace(0, 2, 20)
sml_returns = risk_free_rate + betas * (expected_market_return - risk_free_rate)

plt.figure(figsize=(10,6))
plt.plot(betas, sml_returns, label="Security Market Line")

# scatter plot of assets
for _, row in capm_summary.iterrows():
    plt.scatter(row['Beta'], row['Expected Return (CAPM)'], label=row['Ticker'])

plt.xlabel("Beta")
plt.ylabel("Expected Return")
plt.title("Security Market Line with Multiple Assets")
plt.legend()
plt.grid(True)
plt.show()

# Portfolio weighting

In [ ]:
# Portfolio weights, make sure these sum to 1
weights = {
    "CBA.AX": 0.4,
    "BHP.AX": 0.4,
    "WOW.AX": 0.2
}

capm_summary["Weight"] = capm_summary["Ticker"].map(weights)

portfolio_beta = (capm_summary["Beta"] * capm_summary["Weight"]).sum()
portfolio_expected_return = (capm_summary["Expected Return (CAPM)"] * capm_summary["Weight"]).sum()

In [ ]:
print(f"Portfolio Beta: {portfolio_beta:.4f}")
print(f"Portfolio Expected Return: {portfolio_expected_return:.2%}")

In [ ]:
# add the portfolio dot to the SML
plt.figure(figsize=(10,6))
plt.plot(betas, sml_returns, label="Security Market Line")

# plot the assets
for _, row in capm_summary.iterrows():
    plt.scatter(row['Beta'], row['Expected Return (CAPM)'], label=row['Ticker'])

# add portfolio
plt.scatter(portfolio_beta, portfolio_expected_return, color="black", marker="X", s=100, label="Portfolio")

plt.xlabel("Beta")
plt.ylabel("Expected Return")
plt.title("Security Market Line with Multiple Assets and Portfolio")
plt.legend()
plt.grid(True)
plt.show()

# Add Portfolio Optimisation

In [ ]:
# 1. estimate expected returns (annualized)
mu = expected_returns.mean_historical_return(asset_data)

# 2. estimate sample covariance matrix
S = risk_models.sample_cov(asset_data)

# 3. build the optimizer
ef = EfficientFrontier(mu, S)

# add constraint: no shorting
ef.add_constraint(lambda w: w >= 0)

# add constraint: max 5 non-zero positions
ef.add_constraint(lambda w: w[np.where(w > 0)[0]].shape[0] <= 5)

# maximize Sharpe ratio
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("Optimal Portfolio Weights (max 5 assets):")
print(cleaned_weights)

# portfolio performance
expected_perf = ef.portfolio_performance(verbose=True)

In [ ]:
ef = EfficientFrontier(mu, S)

# constraint: no shorting
ef.add_constraint(lambda w: w >= 0)

# constraint: max 5 nonzero positions
ef.add_constraint(lambda w: w[np.where(w > 0)[0]].shape[0] <= 5)

# maximize Sharpe ratio
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print("Optimal Weights (max 5 stocks):")
print(cleaned_weights)

# portfolio performance
expected_perf = ef.portfolio_performance(verbose=True)

In [ ]:
plotting.plot_efficient_frontier(
    EfficientFrontier(mu, S),
    show_assets=True,
    show_fig=True
)